Loading in Packages 

In [1]:
import numpy as np
import pandas as pd
import os, re, sys
import random

Load in pseq dictionary

In [2]:
path_dict = "~/Work/viralepitope/"
pseq_dict = np.load(path_dict+'pseq_dict_all.npy', allow_pickle = True).item()

MHC allele converted into pseudoseq using dictionary

In [3]:
def convert_alleleseq_to_pseudoseq(seq_dict):
    aa={"A":0,"R":1,"N":2,"D":3,"C":4,"Q":5,"E":6,"G":7,"H":8,"I":9,"L":10,"K":11,"M":12,"F":13,"P":14,"S":15,"T":16,"W":17,"Y":18,"V":19}

    residue_indices = list(range(0,34,1))
    #Now encode the MHC sequences into pseudo-sequences.
    for allele in seq_dict.keys():
        new_pseq = []
        pseq =""
        for index in residue_indices:
            pseq+= seq_dict[allele][index]
            new_pseq.append(seq_dict[allele]) 
        pseq_dict[allele] = new_pseq
    
    return pseq_dict

pseq_dict_converted = convert_alleleseq_to_pseudoseq(pseq_dict)


MHC peptide pair sequence function

In [4]:
def mhc_peptide_pair(path, pseq_dict_matrix):
    aa={"A":0,"R":1,"N":2,"D":3,"C":4,"Q":5,"E":6,"G":7,"H":8,"I":9,"L":10,"K":11,"M":12,"F":13,"P":14,"S":15,"T":16,"W":17,"Y":18,"V":19}
    data_dict = {}
    pep_length = [8,9,10,11,12,13,14,15]
    f = open(path,"r")
    for line in f:
        info = re.split("\t",line)#Retrive information from a tab-delimited line
        allele = info[1].strip()
        if allele in pseq_dict_matrix.keys():
            affinity = int(info[-1].strip()) #Retrive lable information 
            pep = info[0].strip() #Retrive ligand information 
            
            if set(list(pep)).difference(list('ACDEFGHIKLMNPQRSTVWY')):
                print('Illegal peptides')
                continue   
            if len(pep) not in pep_length:
                print('Illegal peptides')
                continue 
                
            new_data = [pep, pseq_dict_matrix[allele], affinity]
            
            if allele not in data_dict.keys():
                data_dict[allele] = [new_data]
            else:
                data_dict[allele].append(new_data)
                
    return data_dict


In [6]:
pseudo_seq = mhc_peptide_pair(path_dict+'fulldata_withlabels_combined.txt', pseq_dict)

Illegal peptides
Illegal peptides
Illegal peptides
Illegal peptides
Illegal peptides
Illegal peptides


In [7]:
len(pseudo_seq)
#check how many pseudoseq we have

121

Extract sequences from data

In [8]:
training_data = []
for allele in pseudo_seq.keys():
    allele_data = pseudo_seq[allele]
    random.shuffle(allele_data)
    allele_data = np.array(allele_data)
    training_data.extend(allele_data)

[all_pep, all_mhc, all_target] = [[i[j] for i in training_data] for j in range(3)]


C:\Users\luffy\AppData\Local\Temp\ipykernel_4968\1456904489.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  allele_data = np.array(allele_data)


Convert to pandas dataframes

In [9]:
all_pepdf = pd.DataFrame(all_pep)
all_mhc_df = pd.DataFrame(all_mhc)
all_target_df = pd.DataFrame(all_target)


In [10]:
all_pepdf = pd.concat((all_pepdf,all_target_df), axis = 1)
all_mhcdf = pd.concat((all_mhc_df[0], all_target_df), axis = 1)

In [11]:
all_pepdf.columns = ['sequence', 'label']
all_mhcdf.columns = ['sequence', 'label']

In [23]:
pos = all_pepdf['label'] == 1
all_pepdf_pos = all_pepdf[pos]
all_pepdf_neg = all_pepdf[~pos]
pos_mhc = all_mhcdf['label'] == 1
all_mhcdf_pos = all_mhcdf[pos_mhc]
all_mhcdf_neg = all_mhcdf[~pos_mhc]

In [24]:
all_pepdf_pos = all_pepdf_pos.reset_index()
all_pepdf_neg = all_pepdf_neg.reset_index()
all_mhcdf_pos = all_mhcdf_pos.reset_index()
all_mhcdf_neg = all_mhcdf_neg.reset_index()

Save Files
Text files for Positives and Negative in Fasta Format
CSV for Full data as sequence for Protein-Bert

In [26]:
ofile = open('viralepitope/positive_viralepitope_sequenceonly.txt', 'w')
for i in range(all_pepdf_pos.shape[0]):
    ofile.write('>Sequence_'+str(i)+'\n'+all_pepdf_pos['sequence'][i]+'\n')
ofile.close()


     

ofile = open('viralepitope/negative_viralepitope_sequenceonly.txt', 'w')
for i in range(all_pepdf_neg.shape[0]):
    ofile.write('>Sequence_'+str(i)+'\n'+all_pepdf_neg['sequence'][i]+'\n')
ofile.close()


     

ofile = open('viralepitope/positive_mhc_sequenceonly.txt', 'w')
for i in range(all_mhcdf_pos.shape[0]):
    ofile.write('>Sequence_'+str(i)+'\n'+all_mhcdf_pos['sequence'][i]+'\n')
ofile.close()


     

ofile = open('viralepitope/negative_mhc_sequenceonly.txt', 'w')
for i in range(all_mhcdf_neg.shape[0]):
    ofile.write('>Sequence_'+str(i)+'\n'+all_mhcdf_neg['sequence'][i]+'\n')
ofile.close()


     

all_pepdf.to_csv('viralepitope/viralepitope_sequenceonly.csv')
all_mhcdf.to_csv('viralepitope/mhcepitope_sequenceonly.csv')
